## **1. Install and import bibraries**


In [1]:
!export LC_ALL="en_US.UTF-8"
!export LD_LIBRARY_PATH="/usr/lib64-nvidia"
!export LIBRARY_PATH="/usr/local/cuda/lib64/stubs"
!ldconfig /usr/lib64-nvidia

import os
import json
import torch
import random
import evaluate
import numpy as np
import pandas as pd
import torch.nn as nn
from torchcrf import CRF
from datasets import Dataset
from transformers import Trainer
from datasets import load_dataset
from collections import namedtuple
from dataclasses import dataclass, field, asdict
from transformers import AutoTokenizer, TrainingArguments
from mamba_ssm.models.mixer_seq_simple import MambaLMHeadModel
from mamba_ssm.utils.hf import load_config_hf, load_state_dict_hf


from IPython.display import clear_output
clear_output()

In [2]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

'cuda'

## **2. dataset**


In [3]:
label2id = {
    'O': 0,
    'B-BODY': 1,
    'I-BODY': 2,
    'B-SYMP': 3,
    'I-SYMP': 4,
    'B-INST': 5,
    'I-INST': 6,
    'B-EXAM': 7,
    'I-EXAM': 8,
    'B-CHEM': 9,
    'I-CHEM': 10,
    'B-DISE': 11,
    'I-DISE': 12,
    'B-DRUG': 13,
    'I-DRUG': 14,
    'B-SUPP': 15,
    'I-SUPP': 16,
    'B-TREAT': 17,
    'I-TREAT': 18,
    'B-TIME': 19,
    'I-TIME': 20
}

id2label = {
    0: 'O',
    1: 'B-BODY',
    2: 'I-BODY',
    3: 'B-SYMP',
    4: 'I-SYMP',
    5: 'B-INST',
    6: 'I-INST',
    7: 'B-EXAM',
    8: 'I-EXAM',
    9: 'B-CHEM',
    10: 'I-CHEM',
    11: 'B-DISE',
    12: 'I-DISE',
    13: 'B-DRUG',
    14: 'I-DRUG',
    15: 'B-SUPP',
    16: 'I-SUPP',
    17: 'B-TREAT',
    18: 'I-TREAT',
    19: 'B-TIME',
    20: 'I-TIME'
}

In [4]:
# train_path = 'data/train.txt'
train_path = 'data/msra_eval.txt'

df_traindata = pd.read_csv(train_path, delimiter='\t', names=["text", "pre_label"], header=None)

df_traindata['label'] = df_traindata['pre_label'].map(label2id)

dataset = Dataset.from_pandas(df_traindata[['text', 'label']])
# dataset = dataset.train_test_split(train_size=0.8, seed=42)
# dataset["validation"] = dataset.pop("test")

dataset

Dataset({
    features: ['text', 'label'],
    num_rows: 92498
})

In [5]:
eval_path = 'data/test.txt'
df_evaldata = pd.read_csv(eval_path, delimiter='\t', names=["text", "pre_label"], header=None)
df_evaldata['label'] = df_evaldata['pre_label'].map(label2id)

datasettest = Dataset.from_pandas(df_evaldata[['text', 'label']])
datasettest

Dataset({
    features: ['text', 'label'],
    num_rows: 121284
})

In [6]:
# dataset["train"][0]

In [7]:
# type(dataset["train"][0]['label'])

In [8]:
# len(dataset["train"])

## **3. Build Custom Mamba Model for Text Classification**


In [9]:
# Mamba 的 config 類引用了這個詞: https://github.com/state-spaces/mamba/blob/main/mamba_ssm/models/config_mamba.py
@dataclass
class MambaConfig:
    d_model: int = 2560
    n_layer: int = 64
    vocab_size: int = 50277 # 50277
    ssm_cfg: dict = field(default_factory=dict)
    rms_norm: bool = True
    residual_in_fp32: bool = True
    fused_add_norm: bool = True
    # pad_vocab_size_multiple: int = 8
    pad_vocab_size_multiple: int = 16
    tie_embeddings = True

    def to_json_string(self):
        return json.dumps(asdict(self))

    def to_dict(self):
        return asdict(self)

In [10]:
labels = ['B-BODY','I-BODY', 'B-SYMP', 'I-SYMP', 'B-INST', 'I-INST', 'B-EXAM', 'I-EXAM','B-CHEM', 'I-CHEM',
          'B-DISE', 'I-DISE', 'B-DRUG', 'I-DRUG', 'B-SUPP', 'I-SUPP', 'B-TREAT', 'I-TREAT', 'B-TIME', 'I-TIME']

In [11]:
class MambaTextClassification(MambaLMHeadModel):
    def __init__(
        self,
        config: MambaConfig,
        initializer_cfg=None,
        device=None,
        dtype=None,
    ) -> None:
        super().__init__(config, initializer_cfg, device, dtype)

        # 使用 MambaClassificationHead 創建一個分類器，輸入大小為 d_model，類號為 len(id2label)。
        self.classification_head = nn.Linear(config.d_model, len(label2id))
        self.crf = CRF(len(label2id), batch_first=False)# batch_first=True ?   
                            
        del self.lm_head

    def forward(self, input_ids, labels=None):
        # 通過原生模型發送input_ids以接收hidden_states。
        hidden_states = self.backbone(input_ids)

        # 取二維hidden_states的平均值，創建具有代表性的 [CLS] 特徵
        mean_hidden_states = hidden_states.mean(dim=1)

        # 將mean_hidden_states通過分類器的頂部來接收logits。
        logits = self.classification_head(mean_hidden_states)
        print('logits:',logits.shape)

        if labels is None:
          ClassificationOutput = namedtuple("ClassificationOutput", ["logits"])
          return ClassificationOutput(logits=logits)
        else:
          ClassificationOutput = namedtuple("ClassificationOutput", ["loss", "logits"])
          
          # emissions = torch.randn(len(input_ids), 5, len(label2id))
          emissions = torch.randn(len(input_ids), len(input_ids), len(label2id))
          print('emissions.shape:', emissions.shape)
    
          loss = -self.crf.forward(emissions, labels, mask=train_dataset['attention_mask'], reduction='mean')
          # loss_fct = nn.CrossEntropyLoss()
          # loss = loss_fct(logits, labels)

          return ClassificationOutput(loss=loss, logits=logits)

    def predict(self, text, tokenizer, id2label=None):
        input_ids = torch.tensor(tokenizer(text)['input_ids'], device=device)[None] # device = 'cuda'
        with torch.no_grad():
          logits = self.forward(input_ids).logits[0]
          label = np.argmax(logits.cpu().numpy())

        if id2label is not None:
          return id2label[label]
        else:
          return label

    @classmethod
    def from_pretrained(cls, pretrained_model_name, device=None, dtype=None, **kwargs):
        # 從之前訓練的模型載入配置。
        config_data = load_config_hf(pretrained_model_name)
        config = MambaConfig(**config_data)

        # 從配置中初始化模型，並將其傳輸到所需的設備和數據類型。
        model = cls(config, device=device, dtype=dtype, **kwargs)

        # 載入以前訓練的模型狀態。
        model_state_dict = load_state_dict_hf(pretrained_model_name, device=device, dtype=dtype)
        model.load_state_dict(model_state_dict, strict=False)

        # 列印出新初始化的嵌入參數。
        print("Newly initialized embedding:", set(model.state_dict().keys()) - set(model_state_dict.keys()))
        return model

In [12]:
# # 從先前訓練的模型載入 Mamba 模型。
model = MambaTextClassification.from_pretrained("state-spaces/mamba-130m")
model.to(device)

# 從 gpt-neox-20b 模型載入 Mamba 模型的分詞器。
# tokenizer = AutoTokenizer.from_pretrained("EleutherAI/gpt-neox-20b")
tokenizer = AutoTokenizer.from_pretrained("p208p2002/llama-traditional-chinese-120M")

# 從 gpt-neox-20b 模型載入 Mamba 模型的分詞器。
tokenizer.pad_token_id = tokenizer.eos_token_id

Newly initialized embedding: {'crf.transitions', 'classification_head.bias', 'classification_head.weight', 'crf.end_transitions', 'crf.start_transitions'}


In [13]:
model

MambaTextClassification(
  (backbone): MixerModel(
    (embedding): Embedding(50280, 768)
    (layers): ModuleList(
      (0-23): 24 x Block(
        (mixer): Mamba(
          (in_proj): Linear(in_features=768, out_features=3072, bias=False)
          (conv1d): Conv1d(1536, 1536, kernel_size=(4,), stride=(1,), padding=(3,), groups=1536)
          (act): SiLU()
          (x_proj): Linear(in_features=1536, out_features=80, bias=False)
          (dt_proj): Linear(in_features=48, out_features=1536, bias=True)
          (out_proj): Linear(in_features=1536, out_features=768, bias=False)
        )
        (norm): RMSNorm()
      )
    )
    (norm_f): RMSNorm()
  )
  (classification_head): Linear(in_features=768, out_features=21, bias=True)
  (crf): CRF(num_tags=21)
)

## **4. Preprocess dataset**


In [14]:
def preprocess_function(examples):
# 建立一個預處理器函數來對文字進行編碼並截斷長度不超過標記最大輸入長度的字串
    samples = tokenizer(examples["text"], truncation=True)
    # 不需要attention_mask
    # 更多細節可以參考mamba的token masking: https://github.com/state-spaces/mamba/issues/49
    # samples.pop('attention_mask')
    return samples

In [15]:
# 執行文字加密
tokenized_dataset = dataset.map(preprocess_function, batched=True)

Map: 100%|██████████| 92498/92498 [00:01<00:00, 83652.86 examples/s] 


In [16]:
# 為隨機函數設定種子
random.seed(42)

# 建立訓練和測試集
train_dataset = tokenized_dataset# ["train"]
# val_dataset = tokenized_dataset["validation"]

# # 建立評估集以在訓練期間進行評估
# # 由於測試集數量較多，僅抽取1%的測試資料集進行評估
# total_samples = len(val_dataset)
# eval_samples = int(0.01 * total_samples)
# eval_indices = random.sample(range(total_samples), eval_samples)
# eval_dataset = val_dataset.select(eval_indices)

In [17]:
# print('total_samples',total_samples)
# print('eval_samples',eval_samples)
# print('eval_indices',eval_indices)
# print('eval_dataset',eval_dataset)

In [18]:
# print('train_dataset', next(iter(train_dataset)))
for i in zip(iter(train_dataset), range(5)):
    print(i)

({'text': '人', 'label': 0, 'input_ids': [1, 10599], 'attention_mask': [1, 1]}, 0)
({'text': '腦', 'label': 1, 'input_ids': [1, 25998, 27671], 'attention_mask': [1, 1, 1]}, 1)
({'text': '的', 'label': 0, 'input_ids': [1, 3611], 'attention_mask': [1, 1]}, 2)
({'text': '松', 'label': 1, 'input_ids': [1, 13140], 'attention_mask': [1, 1]}, 3)
({'text': '果', 'label': 2, 'input_ids': [1, 25998, 26508], 'attention_mask': [1, 1, 1]}, 4)


In [19]:
train_dataset[1]

{'text': '腦',
 'label': 1,
 'input_ids': [1, 25998, 27671],
 'attention_mask': [1, 1, 1]}

In [20]:
train_path = 'data/msra_eval.txt'

df_traindataTEST = pd.read_csv(train_path, delimiter='\t', names=["text", "pre_label"], header=None)

df_traindataTEST['label'] = df_traindataTEST['pre_label'].map(label2id)

datasetTEST = Dataset.from_pandas(df_traindataTEST[['text', "pre_label", "label"]])
datasetTEST = datasetTEST.train_test_split(train_size=0.8, seed=42)


datasetTEST

DatasetDict({
    train: Dataset({
        features: ['text', 'pre_label', 'label'],
        num_rows: 73998
    })
    test: Dataset({
        features: ['text', 'pre_label', 'label'],
        num_rows: 18500
    })
})

In [21]:
tokenized_datasetTEST = datasetTEST.map(preprocess_function, batched=True)

Map: 100%|██████████| 18500/18500 [00:00<00:00, 42958.77 examples/s]


In [22]:
train_datasetTEST = tokenized_datasetTEST["train"]

In [23]:
for i in zip(iter(train_datasetTEST), range(5)):
    print(i)

({'text': '可', 'pre_label': 'O', 'label': 0, 'input_ids': [1, 9147], 'attention_mask': [1, 1]}, 0)
({'text': '股', 'pre_label': 'B-BODY', 'label': 1, 'input_ids': [1, 25998, 27097], 'attention_mask': [1, 1, 1]}, 1)
({'text': '的', 'pre_label': 'O', 'label': 0, 'input_ids': [1, 3611], 'attention_mask': [1, 1]}, 2)
({'text': '癰', 'pre_label': 'B-SYMP', 'label': 3, 'input_ids': [1, 25998, 243, 165, 188], 'attention_mask': [1, 1, 1, 1, 1]}, 3)
({'text': '這', 'pre_label': 'O', 'label': 0, 'input_ids': [1, 15866], 'attention_mask': [1, 1]}, 4)
